In [ ]:
#notebook for neural style transfer


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


style_img = image_loader("./data/images/neural-style/picasso.jpg")
content_img = image_loader("./data/images/neural-style/dancing.jpg")

assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"

In [ ]:
# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize((imsize,imsize)),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


#style_img = image_loader("./data/images/neural-style/picasso.jpg")
#style_img = image_loader('../cnn_subgraph_visualizer/image_data/imagenet_50/input_images/Afghan_hound_10.JPEG')
content_img = image_loader("./data/images/neural-style/dancing.jpg")

assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"

In [ ]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None,save=False):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated
    if save:
        plt.savefig('plots/'+title.replace(' ','_')+'.png')


plt.figure()
imshow(style_img, title='Style Image')

plt.figure()
imshow(content_img, title='Content Image')

In [ ]:
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        #print('Content Loss')
        #print('target shape: %s'%str(self.target.shape))
        #print('input shape: %s'%str(input.shape))
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)


In [ ]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [ ]:
cnn = models.alexnet(pretrained=True).features.to(device).eval()
#cnn = models.alexnet(pretrained=True).features.to(device).eval()
#cnn

In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [ ]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [ ]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

In [ ]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1,
                       content_layers = content_layers_default,style_layers=style_layers_default):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img, 
        content_layers = content_layers, style_layers = style_layers)
    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')

output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img, num_steps=1000)

plt.figure()
imshow(output, title='Content + Style Only')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
#input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')

output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img)

plt.figure()
imshow(output, title='Input=Content + Content + Style')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
#input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')

output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img, content_weight = 0)

plt.figure()
imshow(output, title='Input=Content + Style')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img, style_weight = 0,num_steps=1000)

plt.figure()
imshow(output, title='Content')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img, content_weight = 0,num_steps=1000)

plt.figure()
imshow(output, title='Style')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            style_img, style_img, input_img, style_weight = 0,num_steps=1000)

plt.figure()
imshow(output, title='Content (style image)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img, style_weight = 0,num_steps=2000,content_layers=['conv_10'])

plt.figure()
imshow(output, title='Content (layer 10)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,num_steps=1000,content_layers=['conv_1'])

plt.figure()
imshow(output, title='Style + Content (layer 1)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,num_steps=1000,content_layers=['conv_2'])

plt.figure()
imshow(output, title='Style + Content (layer 2)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,num_steps=1000,content_layers=['conv_3'])

plt.figure()
imshow(output, title='Style + Content (layer 3)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
#input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,num_steps=1000,content_layers=['conv_7'])

plt.figure()
imshow(output, title='Style + Content (layer 7)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,num_steps=1000,content_layers=['conv_10'])

plt.figure()
imshow(output, title='Style + Content (layer 10)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
# if you want to use white noise instead uncomment the below line:
input_img = torch.randn(content_img.data.size(), device=device)




# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')


output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,num_steps=1000,content_layers=['conv_16'])

plt.figure()
imshow(output, title='Style + Content (layer 16)')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

## Invariances

In [ ]:

# def get_invariance_optimizer(input_img):
#     # this line to show that input is a parameter that requires a gradient
#     optimizer = optim.LBFGS([input_img.requires_grad_()])
#     return optimizer

def get_invariance_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.Adam([input_img.requires_grad_()],lr =.04)
    return optimizer

In [ ]:
import numpy as np

class DiffLoss(nn.Module):

    def __init__(self, target,normalize=True,vectorize = False, target_feature_id = None):
        super(DiffLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.normalize = normalize
        #import pdb;pdb.set_trace()
        self.target_feature_id = target_feature_id
        if len(target.shape) == 3:
            self.vectorize = True
        else:
            self.vectorize = vectorize
        self.target = target.detach()
        #self.normalize = normalize

    def forward(self, input):
        #print('Content Loss')
        #print('target shape: %s'%str(self.target.shape))
        #print('input shape: %s'%str(input.shape))
        #self.loss = F.mse_loss(input, self.target)
        #import pdb;pdb.set_trace()
        self.lossf = torch.nn.CosineSimilarity()
        #print(self.target.shape)
        if self.vectorize:
            self.target_flat = torch.flatten(self.target, start_dim=1)
            if self.target_feature_id is None:
                flat_input = torch.flatten(input, start_dim=1)
            else:
                #print('Got Here!')
                flat_input = torch.flatten(input[:,self.target_feature_id,:,:], start_dim=1)
            self.loss = self.lossf(flat_input,self.target_flat)
        else:
            self.loss = self.lossf(input,self.target)
            self.loss = torch.mean(self.loss)
        #if self.normalize:
        #    self.loss = self.loss/np.sqrt(torch.numel(input))*1000
        return input

In [ ]:
# desired depth layers to compute style/content losses :
same_layers_default = ['conv_4']
diff_layers_default = ['conv_1']

def get_invariance_model_and_losses(cnn, normalization_mean, normalization_std, target_img,
                                   same_layers=same_layers_default,diff_layers=diff_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    same_losses = []
    diff_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)
    
    #pixel input losses
    if 'pixel' in same_layers:
        # add sameness loss:
        target = model(target_img).detach()
        same_loss = DiffLoss(target)
        model.add_module("same_loss_pixel", same_loss)
        same_losses.append(same_loss)
        
    if 'pixel' in diff_layers:
        # add sameness loss:
        target = model(target_img).detach()
        diff_loss = DiffLoss(target)
        model.add_module("diff_loss_pixel", diff_loss)
        diff_losses.append(diff_loss)
    
    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)         
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)
        for same_layer in same_layers:
            if name == same_layer or name+':' in same_layer:
                # add sameness loss:
                if ':' not in same_layer:
                    target = model(target_img).detach()
                    same_loss = DiffLoss(target)
                else:
                    feature = int(same_layer.split(':')[-1])
                    target = model(target_img)[:,feature,:,:].detach()
                    same_loss = DiffLoss(target,target_feature_id = feature)
                
                model.add_module("same_loss_{}".format(name), same_loss)
                same_losses.append(same_loss)

        
        for diff_layer in diff_layers:
            if name == diff_layer or name+':' in diff_layer:
                # add difference loss:
                if ':' not in diff_layer:
                    target = model(target_img).detach()
                    diff_loss = DiffLoss(target)
                else:
                    feature = int(diff_layer.split(':')[-1])
                    target = model(target_img)[:,feature,:,:].detach()
                    diff_loss = DiffLoss(target,target_feature_id=feature)

                model.add_module("diff_loss_{}".format(name), diff_loss)
                diff_losses.append(diff_loss)
        

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], DiffLoss):
            break

    model = model[:(i + 1)]
    return model, same_losses, diff_losses

In [ ]:
from copy import deepcopy

def run_invariant_image(cnn, normalization_mean, normalization_std,
                       target_img, input_img, num_steps=300,
                       same_weight=1, diff_weight=1,
                       diff_layers = diff_layers_default,same_layers=same_layers_default,orthog=False):
    """Run the style transfer."""
    print('Building the model..')
    model, same_losses, diff_losses = get_invariance_model_and_losses(cnn,
        normalization_mean, normalization_std, target_img, 
        same_layers = same_layers, diff_layers = diff_layers)
    optimizer = get_invariance_optimizer(input_img)

    
    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            diff_score = 0
            same_score = 0
            
            for dl in diff_losses:
                if orthog:
                    diff_score += torch.abs(dl.loss)
                else:
                    diff_score += dl.loss
            for sl in same_losses:
                same_score += sl.loss
                
            same_score /= float(len(same_losses))
            diff_score /= float(len(diff_losses))
        
            
            diff_score *= diff_weight
            same_score *= same_weight
            
            
            #below is really what you have to mess with
            loss = diff_score - same_score
            loss.backward()

            run[0] += 1
            if run[0] % 100 == 0 or run[0] == 1:
                print("run {}:".format(run))
                print('Diff Loss : {:4f} Same Loss: {:4f}'.format(
                    diff_score.item(), same_score.item()))
                

            return diff_score-same_score

        optimizer.step(closure)
        #if run[0] % 50 == 0:
        #    plt.figure()
        #    imshow(input_img)

        #    plt.ioff()
        #    plt.show()
            

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
# if you want to use white noise instead uncomment the below line:
image_name = '../cnn_subgraph_visualizer/image_data/imagenet_50/input_images/Afghan_hound_10.JPEG'
content_img = image_loader(image_name)
#input_img = torch.randn(content_img.data.size(), device=device)
input_img = content_img.clone() + torch.randn(content_img.data.size(), device=device)*.01
#input_img = content_img.clone()
target_img = content_img.clone()
#target_img = torch.randn(content_img.data.size(), device=device)


# add the original input image to the figure:

plt.figure()
imshow(input_img, title='Input Image')


output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                            target_img, deepcopy(input_img),
                             num_steps=300,
                             diff_layers = ['conv_1:1'],
                             same_layers = ['conv_1:2'],
                            same_weight=1,orthog=True)

plt.figure()
imshow(input_img,'Input Img')

plt.figure()
imshow(target_img,'Target Img')

plt.figure()
imshow(output,'Output Img')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()


output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                            target_img, deepcopy(input_img),
                             num_steps=300,
                             diff_layers = ['conv_1:1'],
                             same_layers = ['conv_1:2'],
                            same_weight=1,orthog=False)

plt.figure()
imshow(input_img,'Input Img')

plt.figure()
imshow(target_img,'Target Img')

plt.figure()
imshow(output,'Output Img')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
contrasts = [
    ('pixel','conv_1'),
    ('pixel','conv_2'),
    ('pixel','conv_3'),
    ('pixel','conv_4'),
    ('pixel','conv_5'),
    ('pixel','conv_6'),
    ('pixel','conv_7'),
    ('pixel','conv_8'),
    ('pixel','conv_9'),
    ('pixel','conv_10'),
    ('pixel','conv_11'),
    ('pixel','conv_12'),
    ('pixel','conv_13'),
    ('pixel','conv_14'),
    ('pixel','conv_15'),
    ('pixel','conv_16'),
    ('conv_1','pixel'),
    ('conv_2','pixel'),
    ('conv_3','pixel'),
    ('conv_4','pixel'),
    ('conv_5','pixel'),
    ('conv_6','pixel'),
    ('conv_7','pixel'),
    ('conv_8','pixel'),
    ('conv_9','pixel'),
    ('conv_10','pixel'),
    ('conv_11','pixel'),
    ('conv_12','pixel'),
    ('conv_13','pixel'),
    ('conv_14','pixel'),
    ('conv_15','pixel'),
    ('conv_16','pixel'),
    ('conv_17','pixel'),
    ('conv_18','pixel'),
    ('conv_19','pixel'),
    ('conv_20','pixel')
]



# if you want to use white noise instead uncomment the below line:
image_name = '../cnn_subgraph_visualizer/image_data/imagenet_50/input_images/Afghan_hound_10.JPEG'
content_img = image_loader(image_name)
#input_img = torch.randn(content_img.data.size(), device=device)
input_img = content_img.clone() + torch.randn(content_img.data.size(), device=device)*.01
#input_img = content_img.clone()
target_img = content_img.clone()
#target_img = torch.randn(content_img.data.size(), device=device)


# add the original input image to the figure:

plt.figure()
imshow(input_img, title='Input Image')


for contrast in contrasts:
    output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                            target_img, deepcopy(input_img),
                             num_steps=2000,
                             diff_layers = [contrast[0]],
                             same_layers = [contrast[1]],
                            same_weight=1)

    plt.figure()
    imshow(input_img,'Input Img')

    plt.figure()
    imshow(target_img,'Target Img')

    plt.figure()
    imshow(output,'Difference: %s Same: %s'%(contrast[0],contrast[1]))

    # sphinx_gallery_thumbnail_number = 4
    plt.ioff()
    plt.show()

In [ ]:
x=torch.randn(content_img.data.size(), device=device)
y = torch.flatten(x, start_dim=1)
y.shape

In [ ]:
contrasts = [
    ('conv_1','conv_2'),
    ('conv_2','conv_3'),
    ('conv_3','conv_4'),
    ('conv_4','conv_4'),
    ('conv_5','conv_6'),
    ('conv_6','conv_7'),
    ('conv_7','conv_8'),
    ('conv_8','conv_9'),
    ('conv_9','conv_10'),
    ('conv_10','conv_11'),
    ('conv_11','conv_12'),
    ('conv_12','conv_13'),
    ('conv_13','conv_14'),
    ('conv_14','conv_15'),
    ('conv_15','conv_16'),
    ('conv_2','conv_1'),
    ('conv_3','conv_2'),
    ('conv_4','conv_3'),
    ('conv_5','conv_4'),
    ('conv_6','conv_5'),
    ('conv_7','conv_6'),
    ('conv_8','conv_7'),
    ('conv_9','conv_8'),
    ('conv_10','conv_9'),
    ('conv_11','conv_10'),
    ('conv_12','conv_11'),
    ('conv_13','conv_12'),
    ('conv_14','conv_13'),
    ('conv_15','conv_14'),
    ('conv_16','conv_15'),
]



# if you want to use white noise instead uncomment the below line:
image_name = '../cnn_subgraph_visualizer/image_data/imagenet_50/input_images/Afghan_hound_10.JPEG'
content_img = image_loader(image_name)
#input_img = torch.randn(content_img.data.size(), device=device)
input_img = content_img.clone() + torch.randn(content_img.data.size(), device=device)*.01
#input_img = content_img.clone()
target_img = content_img.clone()
#target_img = torch.randn(content_img.data.size(), device=device)


# add the original input image to the figure:

plt.figure()
imshow(input_img, title='Input Image')


for contrast in contrasts:
    output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                            target_img, deepcopy(input_img),
                             num_steps=2000,
                             diff_layers = [contrast[0]],
                             same_layers = [contrast[1]],
                            same_weight=1)

    plt.figure()
    imshow(input_img,'Input Img')

    plt.figure()
    imshow(target_img,'Target Img')

    plt.figure()
    imshow(output,'Difference: %s Same: %s'%(contrast[0],contrast[1]))

    # sphinx_gallery_thumbnail_number = 4
    plt.ioff()
    plt.show()

In [ ]:
contrasts = [
#     ('conv_1','conv_3'),
#     ('conv_1','conv_4'),
#     ('conv_1','conv_6'),
#     ('conv_1','conv_8'),
#     ('conv_1','conv_10'),
#     ('conv_1','conv_12'),
#     ('conv_1','conv_14'),
#     ('conv_1','conv_16'),
#     ('conv_16','conv_1'),
#     ('conv_14','conv_1'),
#     ('conv_12','conv_1'),
#     ('conv_10','conv_1'),
#     ('conv_8','conv_1'),
#     ('conv_6','conv_1'),
#     ('conv_4','conv_1'),
#     ('conv_3','conv_1'),
#     ('conv_4','conv_14'),
#     ('conv_14','conv_4'),
      ('relu_1:2','relu_14:2'),
      ('conv_1','conv_14:2'),
      ('conv_14:2','conv_1:2'),
      ('conv_14:2','conv_1')
      
     
 ]
contrasts = []
for i in ['1','2','3']:
    for j in ['1','2','3']:
        for l1 in ['1','2','4','7','12','16']:
            for l2 in ['1','2','4','7','12','16']:
                if l1 == l2 and i ==j:
                    continue
                contrasts.append(('conv_%s:%s'%(i,l1),'conv_%s:%s'%(j,l2)))


# if you want to use white noise instead uncomment the below line:
image_name = '../cnn_subgraph_visualizer/image_data/imagenet_50/input_images/Afghan_hound_10.JPEG'
content_img = image_loader(image_name)
#input_img = torch.randn(content_img.data.size(), device=device)
input_img = content_img.clone() + torch.randn(content_img.data.size(), device=device)*.01
#input_img = content_img.clone()
target_img = content_img.clone()
#target_img = torch.randn(content_img.data.size(), device=device)


# add the original input image to the figure:

plt.figure()
imshow(input_img, title='Input Image')


for contrast in contrasts:
    output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                            target_img, deepcopy(input_img),
                             num_steps=500,
                             diff_layers = [contrast[0]],
                             same_layers = [contrast[1]],
                            same_weight=1)

    #plt.figure()
    #imshow(input_img,'Input Img')

    plt.figure()
    imshow(target_img,'Target Img')
    
    title = 'Difference: %s Same: %s'%(contrast[0],contrast[1])
    plt.figure()
    imshow(output,'Different: %s Same: %s'%(contrast[0],contrast[1]),save=True)
    #plt.savefig('plots/'+title+'.png')

    # sphinx_gallery_thumbnail_number = 4
    plt.ioff()
    plt.show()

In [ ]:
# contrasts = [
# #     ('conv_1','conv_3'),
# #     ('conv_1','conv_4'),
# #     ('conv_1','conv_6'),
# #     ('conv_1','conv_8'),
# #     ('conv_1','conv_10'),
# #     ('conv_1','conv_12'),
# #     ('conv_1','conv_14'),
# #     ('conv_1','conv_16'),
# #     ('conv_16','conv_1'),
# #     ('conv_14','conv_1'),
# #     ('conv_12','conv_1'),
# #     ('conv_10','conv_1'),
# #     ('conv_8','conv_1'),
# #     ('conv_6','conv_1'),
# #     ('conv_4','conv_1'),
# #     ('conv_3','conv_1'),
# #     ('conv_4','conv_14'),
# #     ('conv_14','conv_4'),
#       ('relu_1:2','relu_14:2'),
#       ('conv_1','conv_14:2'),
#       ('conv_14:2','conv_1:2'),
#       ('conv_14:2','conv_1')
      
     
#  ]
# contrasts = []
# for i in ['1','2','3']:
#     for j in ['1','2','3']:
#         for l1 in ['1','2','4','7','12','16']:
#             for l2 in ['1','2','4','7','12','16']:
#                 if l1 == l2 and i ==j:
#                     continue
#                 contrasts.append(('conv_%s:%s'%(i,l1),'conv_%s:%s'%(j,l2)))
contrasts = []
for i in range(64):
    contrasts.append([['conv_1:%s'%str(i)],['conv_1']])

# if you want to use white noise instead uncomment the below line:
image_name = '../cnn_subgraph_visualizer/image_data/imagenet_50/input_images/Afghan_hound_10.JPEG'
content_img = image_loader(image_name)
#input_img = torch.randn(content_img.data.size(), device=device)
input_img = content_img.clone() + torch.randn(content_img.data.size(), device=device)*.01
#input_img = content_img.clone()
target_img = content_img.clone()
#target_img = torch.randn(content_img.data.size(), device=device)


# add the original input image to the figure:

plt.figure()
imshow(input_img, title='Input Image')

for contrast in contrasts:
    output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                            target_img, deepcopy(input_img),
                             num_steps=500,
                             diff_layers = contrast[0],
                             same_layers = contrast[1],
                            same_weight=1,orthog=True)

    #plt.figure()
    #imshow(input_img,'Input Img')

    plt.figure()
    imshow(target_img,'Target Img')

    title = 'Difference: %s Same: %s Orthog'%(contrast[0],contrast[1])
    plt.figure()
    imshow(output,title,save=True)
    #plt.savefig('plots/'+title+'.png')

    # sphinx_gallery_thumbnail_number = 4
    plt.ioff()
    plt.show()
    
    output = run_invariant_image(cnn, cnn_normalization_mean, cnn_normalization_std,
                        target_img, deepcopy(input_img),
                         num_steps=500,
                         diff_layers = contrast[0],
                         same_layers = contrast[1],
                        same_weight=1,orthog=False)
    #plt.figure()
    #imshow(input_img,'Input Img')

    plt.figure()
    imshow(target_img,'Target Img')

    title = 'Difference: %s Same: %s'%(contrast[0],contrast[1])
    plt.figure()
    imshow(output,title,save=True)
    #plt.savefig('plots/'+title+'.png')

    # sphinx_gallery_thumbnail_number = 4
    plt.ioff()
    plt.show()